In [1]:
import sys
import importlib
sys.path.append(r'../../pykneer')
sys.path.append(r'../pykneer')
import pykneer_io as io
importlib.reload(io)
import preprocessing_for_nb as prep
importlib.reload(prep)
import segmentation_sa_for_nb as segm
importlib.reload(segm)

import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append("../../..")
from MyProcessDicom import *

from preparing_images import prepare_image_and_list, prepare_centerline
from display_and_show_images import *

In [2]:
def load_data(data_type):
    if data_type == "newsubject":
        input_file_name = "./image_list_newsubject.txt"
        modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
    elif data_type == "reference":
        input_file_name = "./image_list_reference.txt"
        modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal"
    elif data_type == "longitudinal":
        input_file_name = "./image_list_longitudinal.txt"
        modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
    
    n_of_cores      = 1
    image_data = io.load_image_data_segmentation(modality, input_file_name)
    return image_data

In [15]:
# process list
# first in TP as reference
caselist = [
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
]

# prepare_image_and_list(caselist)
prepare_centerline(caselist)

ahaknee18tp29  exist
ahaknee24tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee48tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee72tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee18tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist


In [29]:
def modify_center_line(all_image_data):
    for i in range(0, len(all_image_data)):
        image_data                    = all_image_data[i]
   
        file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg/reference/centerline/" + image_data['image_name_root'] + '_line.txt'
#         image_data['image_name_root']  image_data['fmask'][:-11]
        ref_path = file_name
        # change laterality
        laterality = image_data["laterality"]
        if laterality == "right" or laterality == "Right":
            df = pd.read_csv(file_name, header=1, sep=' ', names=['x','y','z'])
            line = np.array([df['x'], df['y'], df['z']]).T
            df['x'] = 384 - df['x']
            with open(ref_path,"w+") as f:
                f.write("point\n")
                f.write(str(line.shape[0])+"\n")
            df.to_csv(ref_path,sep=" ", mode='a', header=None, index=False)
    return

input_file_name           = ".\image_list_preprocessing.txt"
n_of_cores                = 1 # change the number of cores according to your computer
intensity_standardization = 0

# Image information
image_data = io.load_image_data_preprocessing(input_file_name)
# image_data = load_data("reference")
modify_center_line(image_data)

-> image information loaded


In [4]:
def preprocess():
    input_file_name           = ".\image_list_preprocessing.txt"
    n_of_cores                = 1 # change the number of cores according to your computer
    intensity_standardization = 0

    # Image information
    image_data = io.load_image_data_preprocessing(input_file_name)

    ref = sitk.ReadImage('../../reference/newsubject/reference.mha')

    # Spatial standardization
    # Step 0: Read the dicom images
    prep.read_dicom_stack(image_data, n_of_cores)
    # Save image header
    prep.print_dicom_header(image_data, n_of_cores)    # save to *.txt

    # Step 1: Change orientation to RAI
    #prep.axial_to_sagittal(image_data, n_of_cores)
    prep.orientation_to_rai(image_data, n_of_cores)

    # Step 2: Change laterality if knee is right
    prep.flip_rl(image_data, n_of_cores)
    # Step 3: Set image origin to (0,0,0)
    prep.origin_to_zero(image_data, n_of_cores)    # Images are saved as *_orig.mha and they are anonymized

    # 4. Correct magnetic field inhomogeneities
    # Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. 
    # This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop
    if intensity_standardization == 1:
        prep.field_correction(image_data, n_of_cores)
        prep.rescale_to_range(image_data, n_of_cores)  # 5. Rescale intensities to [0 100]
        prep.edge_preserving_smoothing(image_data, n_of_cores)  # 6. Edge preserving smoothing

# preprocess()

In [3]:
def segmentation_sa():
    input_file_name = "./image_list_newsubject.txt"
    modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
    segm.prepare_reference(image_data)
    ### NOTE!!! reference_fv is dummy!!!

    # Segment Bone
#     # 1. Register image to reference
    segm.register_bone_to_reference(image_data, n_of_cores)
#     # 2. Invert transformations
    segm.invert_bone_transformations(image_data, n_of_cores)
#     # 3. Warp reference mask to moving image
    segm.warp_bone_mask(image_data, n_of_cores)
    
    segm.register_tibia_to_reference(image_data, n_of_cores)
    segm.invert_tibia_transformations(image_data, n_of_cores)
    segm.warp_tibia_mask(image_data, n_of_cores)

    # Segment cartilage
    segm.register_cartilage_to_reference(image_data, n_of_cores)
    segm.invert_cartilage_transformations(image_data, n_of_cores)
    segm.warp_cartilage_mask(image_data, n_of_cores)
    
# segmentation_sa()

In [10]:
image_data = load_data("newsubject")

-> image information loaded


In [4]:
image_data = load_data("longitudinal")

-> image information loaded


In [6]:
image_data[0]

{'registration_type': 'longitudinal',
 'vessel': 'fv',
 'cartilage': 'fc',
 'bone': 'f',
 'current_anatomy': [],
 'reference_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\reference\\longitudinal\\',
 'reference_name': '9189303L_TP0_prep.mha',
 'reference_root': '9189303L_TP0_prep',
 'moving_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\preprocessed\\',
 'moving_name': '9189303L_TP1_prep.mha',
 'moving_root': '9189303L_TP1_prep',
 'registered_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\registered\\',
 'segmented_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\segmented\\',
 'registered_sub_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\registered\\9189303L_TP1_prep\\',
 'i_registered_sub_folder': 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\registered\\9189303L_TP1_prep\\invert\\',
 'dilate_radius': 15,
 'fmask_file_name': '9189303L_TP0_prep_f.mha',
 'fdil_mask_file_name': '9189303L_TP0_prep_f_15.mha',
 'flevelset_mask_file_name': '9189303L_TP0_prep_f_le

In [25]:
def segmentation_sa_long():
    input_file_name = "./image_list_longitudinal.txt"
    modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
    segm.prepare_reference(image_data)

#     segm.register_bone_to_reference(image_data, n_of_cores)
#     segm.invert_bone_transformations(image_data, n_of_cores)
#     segm.warp_tibia_mask(image_data, n_of_cores)
#     segm.warp_bone_mask(image_data, n_of_cores)
    
    segm.warp_centerline(image_data, n_of_cores)

    # segm.register_vessel_to_reference(image_data, n_of_cores)
    # segm.invert_vessel_transformations(image_data, n_of_cores)
    # segm.warp_vessel_mask(image_data, n_of_cores)
    
#     segm.register_cartilage_to_reference(image_data, n_of_cores)
#     segm.invert_cartilage_transformations(image_data, n_of_cores)
#     segm.warp_cartilage_mask(image_data, n_of_cores)
    
segmentation_sa_long()

-> image information loaded
9189303L_TP0_prep.mha
9189303R_TP0_prep.mha
9189303R_TP0_prep.mha
9189303R_TP0_prep.mha
9189303R_TP0_prep.mha
9189303R_TP0_prep.mha
9939718L_TP0_prep.mha
9939718L_TP0_prep.mha
9939718L_TP0_prep.mha
9939718L_TP0_prep.mha
9939718L_TP0_prep.mha
9939718L_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9939718R_TP0_prep.mha
9941231L_TP0_prep.mha
9941231L_TP0_prep.mha
9941231L_TP0_prep.mha
9941231L_TP0_prep.mha
9941231L_TP0_prep.mha
9941231L_TP0_prep.mha
9941231R_TP0_prep.mha
9941231R_TP0_prep.mha
9941231R_TP0_prep.mha
9941231R_TP0_prep.mha
9942724L_TP0_prep.mha
9942724L_TP0_prep.mha
9942724L_TP0_prep.mha
9942724L_TP0_prep.mha
9942724R_TP0_prep.mha
9942724R_TP0_prep.mha
9942724R_TP0_prep.mha
9942724R_TP0_prep.mha
9942724R_TP0_prep.mha
9941446L_TP0_prep.mha
9941446L_TP0_prep.mha
9941446L_TP0_prep.mha
9941446L_TP0_prep.mha
9941446L_TP0_prep.mha
9941446R_TP0_prep.mha
9941

In [6]:
# preprocess()
# segmentation_sa()
# segmentation_sa_long()

In [12]:
image_data = load_data("longitudinal")

-> image information loaded


In [ ]:
image_data[0]

In [62]:
# mask_file_name = image_data[0]['reference_folder'] + '9941231L_TP0_prep_f.mha'
mask_file_name = image_data[0]['segmented_folder'] + '9404954L_TP3_prep_f.mha'
# mask_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\newsubject\\reference_f.mha"
tibia_file_name = image_data[0]['segmented_folder'] + '9404954L_TP3_prep_t_rigid.mha'
# tibia_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9404954L_TP3_prep_t.mha"
# mask_file_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9941446L_TP0_prep_t.mha'
moving_file_name   = image_data[0]['moving_folder']    + "9404954L_TP3_prep.mha"
# moving_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\registered\\9404954L_TP1_prep\\t_rigid.mha"
mask = sitk.ReadImage(mask_file_name)
tibia = sitk.ReadImage(tibia_file_name)
moving = sitk.ReadImage(moving_file_name)
key = 1
if key == 1:
    mask = resample_bwimage(mask)
    tibia = resample_bwimage(tibia)
    moving = resample_bwimage(moving)
#     show_masked_images_sagittal(moving, mask)
    show_all_masked_images(moving, mask,tibia)
else:
#     show_all_masked_images(moving, mask,tibia)
    show_masked_images(moving, tibia)    

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [7]:
image_data = image_data[0]
reference_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\" + image_data['fmask'][:-5] + 't.mha'
tibia_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 't_rigid.mha'
ref = sitk.ReadImage(reference_file_name)
tibia = sitk.ReadImage(tibia_file_name)

In [6]:
show_sagital_reference(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10, 9939718R_TP10, 9941231L_TP10, 9942724L_TP10, 9942724R_TP10,
# 9941446R_TP1, 9404954L_TP3, 9404954L_TP10

In [14]:
show_sagital_slice_images(image_data)

9189303L_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9939718L_TP0


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030L_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [16]:
# rigid_only
show_sagital_slice_images(image_data, is_rigid=True)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10, 9941231L_TP10, 9942723L_TP10, 9942724R_TP10

In [20]:
show_sagital_slice_images(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10

In [4]:
def get_max_coordinate(image):
    array = sitk.GetArrayFromImage(image)
    return np.max(np.nonzero(np.argmax(np.argmax(array, axis = 1), axis=1)))

def computeQualityMeasures(lP,lT):
    quality=dict()
#     labelPred=sitk.GetImageFromArray(lP, isVector=False)
#     labelTrue=sitk.GetImageFromArray(lT, isVector=False)
    labelPred = lP
    labelTrue = lT
    space = labelPred.GetSpacing()[2]
    hausdorffcomputer=sitk.HausdorffDistanceImageFilter()
    hausdorffcomputer.Execute(labelTrue,labelPred)
    quality["avgHausdorff"]=hausdorffcomputer.GetAverageHausdorffDistance() * space
    quality["Hausdorff"]=hausdorffcomputer.GetHausdorffDistance() * space
 
    dicecomputer=sitk.LabelOverlapMeasuresImageFilter()
    dicecomputer.Execute(labelTrue,labelPred)
    quality["dice"]=dicecomputer.GetDiceCoefficient()
    
    quality["max_y_coordinate"]= (get_max_coordinate(labelPred) - get_max_coordinate(labelTrue)) * space
 
    return quality

In [9]:
def compute_quality_metrics(all_image_data):
    last_name = '0000000L'
    avgHausdorff, Hausdorff, dice, max_y_coordinate, count = {}, {}, {}, {}, {}
    # for each image
    for i in range(0, len(all_image_data)):
        image_data                    = all_image_data[i]
        tpid = image_data['moving_name'][9:12]
        if(image_data['moving_name'][12]=="_"):
#             print(image_data['moving_name'][:12])
            pass
        else:
            tpid += image_data['moving_name'][12]
#             print(image_data['moving_name'][:13])
   
        reference_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\" + image_data['fmask'][:-5] + 't.mha'
        tibia_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 't_rigid.mha'
#         moving_file_name   = image_data['moving_folder']    + image_data['moving_name']
        ref = sitk.ReadImage(reference_file_name)
        tibia = sitk.ReadImage(tibia_file_name)
        ref = resample_bwimage(ref)
        tibia = resample_bwimage(tibia)
        quality = computeQualityMeasures(tibia,ref)
        avgHausdorff[tpid] = avgHausdorff.get(tpid, 0) + quality['avgHausdorff']
        Hausdorff[tpid] = Hausdorff.get(tpid, 0) + quality['Hausdorff']
        dice[tpid] = dice.get(tpid, 0) + quality['dice']
        max_y_coordinate[tpid] = max_y_coordinate.get(tpid, 0) + abs(quality['max_y_coordinate'])
        count[tpid] = count.get(tpid, 0) + 1
#         print(quality)
    print( {'avgHausdorff': avgHausdorff, 'Hausdorff': Hausdorff, 'dice': dice, 'max_y_coordinate': max_y_coordinate, 'count': count} )

In [10]:
compute_quality_metrics(image_data)

{'avgHausdorff': {'TP1': 0.7118573590349062, 'TP3': 4.004194205804441, 'TP5': 2.537208439082805, 'TP6': 1.4040581590037187, 'TP8': 0.8927426719157536, 'TP10': 8.736262033499571, 'TP2': 0.12719809163426452, 'TP4': 0.03795664890830886}, 'Hausdorff': {'TP1': 28.871873847015067, 'TP3': 43.84979638427557, 'TP5': 48.24602779706839, 'TP6': 41.17499453583541, 'TP8': 29.877543056246658, 'TP10': 66.37907188318565, 'TP2': 3.062951281048318, 'TP4': 2.355361937042857}, 'dice': {'TP1': 13.176092973249032, 'TP3': 11.030711078887368, 'TP5': 8.658368570368053, 'TP6': 11.723608414664602, 'TP8': 7.0889394786701665, 'TP10': 5.782924064639946, 'TP2': 0.8789365857165107, 'TP4': 0.9160871925816422}, 'max_y_coordinate': {'TP1': 5.833328, 'TP3': 29.895806, 'TP5': 1.458332, 'TP6': 5.833328, 'TP8': 2.916664, 'TP10': 55.781198999999994, 'TP2': 0.0, 'TP4': 1.458332}, 'count': {'TP1': 14, 'TP3': 13, 'TP5': 10, 'TP6': 13, 'TP8': 8, 'TP10': 9, 'TP2': 1, 'TP4': 1}}


In [18]:
from math import sqrt
import pandas as pd
def computeCenterLineMeasures(ref_path, pred_path):
    df = pd.read_csv(pred_path, header=None, sep=' ', names=['x','y','z'])
    line = np.array([df['x'], df['y'], df['z']]).T
    df2 = pd.read_csv(ref_path, header=1, sep=' ', names=['x','y','z'])
    line2 = np.array([df2['x'], df2['y'], df2['z']]).T

    j = 0
    dist, cnt = 0, 0
    for i in range(line.shape[0]):
        tmp_z = line[i][2]
        if tmp_z < line2[j][2]:
            continue
        while tmp_z >= line2[j][2] and j < line2.shape[0]-1:
            j = j + 1
        if j >= line2.shape[0]:
            break
        if (line2[j][2] - tmp_z) < (tmp_z - line2[j-1][2]):
            kk = j
        else:
            kk = j - 1
        dist += sqrt((line[i][0] - line2[kk][0])**2 + (line[i][1] - line2[kk][1])**2)
        cnt += 1
    print(dist/cnt, cnt)
    return dist/cnt

In [19]:
def compute_centerline_metrics(all_image_data):
    for i in range(0, len(all_image_data)):
        image_data                    = all_image_data[i]
        tpid = image_data['moving_name'][9:12]
        if(image_data['moving_name'][12]=="_"):
            print(image_data['moving_name'][:12], end = ':\t')
        else:
            tpid += image_data['moving_name'][12]
            print(image_data['moving_name'][:13], end = ':\t')
   
        reference_line_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\centerline\\" + image_data['fmask'][:-10] + 'line.txt'
        predict_line_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\centerline\\' + image_data['fmask'][:-10] + 'line.txt'
#         moving_file_name   = image_data['moving_folder']    + image_data['moving_name']
        
        dist = computeCenterLineMeasures(reference_line_name, predict_line_name)
    return

In [23]:
compute_centerline_metrics(image_data)

9189303L_TP1:	13.174734192473478 74
9189303R_TP1:	3.840812512547267 75
9189303R_TP3:	7.515185089856094 75
9189303R_TP5:	11.303035682844406 75
9189303R_TP6:	10.686502317176782 75
9189303R_TP8:	10.27270441417675 75
9939718L_TP1:	9.621002503870901 60
9939718L_TP3:	9.88255216910765 60
9939718L_TP5:	14.051994422410349 60
9939718L_TP6:	21.251567936851636 60
9939718L_TP8:	15.898434553176042 60
9939718L_TP10:	35.3311579816151 60
9939718R_TP1:	8.840404192041737 60
9939718R_TP2:	7.5694820201787145 60
9939718R_TP3:	8.829782197083777 60
9939718R_TP5:	16.4561737515423 60
9939718R_TP6:	17.58056002514159 60
9939718R_TP8:	8.673203803691706 60
9939718R_TP10:	11.896237953546661 60
9941231L_TP1:	7.856329469000292 69
9941231L_TP3:	8.151868722985785 69
9941231L_TP5:	24.955421116006164 68
9941231L_TP6:	8.444838663461086 69
9941231L_TP8:	8.70038892813295 69
9941231L_TP10:	39.12525767542819 69
9941231R_TP1:	19.599360408105202 74
9941231R_TP3:	7.0944925541628745 74
9941231R_TP5:	16.01778481878573 74
9941231R_T

In [28]:
compute_centerline_metrics(image_data)

9189303L_TP1:	6.400082624302761 73
9189303R_TP1:	4.942766182320849 75
9189303R_TP3:	13.323938071361104 72
9189303R_TP5:	9.881951505109049 75
9189303R_TP6:	6.607249197529989 75
9189303R_TP8:	8.393954826120256 74
9939718L_TP1:	8.780723984077525 49
9939718L_TP3:	8.450764785378924 55
9939718L_TP5:	20.662835405986076 60
9939718L_TP6:	22.267882119650736 60
9939718L_TP8:	17.841039012752173 41
9939718L_TP10:	33.99653603477443 59
9939718R_TP1:	7.8650528746707185 60
9939718R_TP2:	9.189329212587996 60
9939718R_TP3:	10.55207514745165 60
9939718R_TP5:	25.430747118534587 59
9939718R_TP6:	13.520613625179038 60
9939718R_TP8:	25.56783205618552 60
9939718R_TP10:	42.58154815927226 41
9941231L_TP1:	4.082030734219041 66
9941231L_TP3:	8.291553941775069 69
9941231L_TP5:	23.039244025967108 68
9941231L_TP6:	7.450541451485187 69
9941231L_TP8:	81.79489654197907 55
9941231L_TP10:	42.615297947735655 61
9941231R_TP1:	22.095750063912917 64
9941231R_TP3:	9.735724099847651 72
9941231R_TP5:	19.072527322702726 74
994123

In [27]:
image_data = load_data("longitudinal")

-> image information loaded


In [10]:
show_slice_images(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9189303R_TP1


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9189303R_TP3


...

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9942030R_TP5


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9942030R_TP6


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [50]:
def tmp_resample(fixed_image, reference):
    isoresample = sitk.ResampleImageFilter()
    isoresample.SetInterpolator(sitk.sitkBSpline)
    isoresample.SetOutputDirection(fixed_image.GetDirection())
    isoresample.SetOutputOrigin(fixed_image.GetOrigin())
    orig_spacing = fixed_image.GetSpacing()
    new_spacing = (orig_spacing[0],orig_spacing[0],orig_spacing[0])
    isoresample.SetOutputSpacing(new_spacing)
    orig_size = np.array(fixed_image.GetSize(), dtype=np.int)    
    new_size = orig_size.copy()
    new_size[2] = int(orig_size[2]*(orig_spacing[2]/orig_spacing[0])+0.5)
    new_size = [int(s) for s in new_size]
    print(orig_size, new_size)
    isoresample.SetSize(new_size)
    return isoresample.Execute(fixed_image)

In [29]:
def show_registered_images(find_str): 
    path = 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\registered\\'
    file_path_list = []
    reference = sitk.ReadImage(path+find_str+"_TP0_prep\\f_rigid.mha")
    for file in os.listdir(path):
        if find_str in file:
            file_path_list.append(file)
    i = 1
    # Create a figure with two subplots and the specified size.
    plt.subplots(len(file_path_list), 1,figsize=(25,20))
    for tmp_path in file_path_list:
        result_file_name              = path + tmp_path + '\\f_rigid.mha'
#         mask_file_name                = image_data["segmented_folder"] + image_data[anatomy + "mask"]

        # read the images
        result = sitk.ReadImage(result_file_name)
#         mask   = sitk.ReadImage(mask_file_name)

        # images from simpleitk to numpy
    #     result_py = sitk.GetArrayFromImage(result)
    #         mask_py   = sitk.GetArrayFromImage(mask)
        pos1 = tmp_path.find('TP', 0)
        pos2 = tmp_path.find('_', pos1)
        tp = tmp_path[pos1:pos2]
        if tp != 'TP0':
            result = resample_image(result)
        
            
        show_sagittal_images(result, len(file_path_list), i, tp)
        i = i + 1
    plt.show() 

# show_registered_images('9939718R')

In [30]:
def display_sagittal_images(fixed_image, moving_image):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    fixed_npa = sitk.GetArrayViewFromImage(fixed_image)
    moving_npa = sitk.GetArrayViewFromImage(moving_image)
    size = np.size(fixed_npa, 2)
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[:,:,size//2],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[:,:,size//2],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.show() 